### Notebook: Cleaning

In this notebook, we'll take care of the cleaning of the Evides and the Fixed Supply Points datasets. We'll start with Evides.

In [ ]:
# Import libraries 
import pandas as pd
import numpy as np
import jellyfish
pd.options.mode.chained_assignment = None

# Import data
evides = pd.read_csv('evides_2022.csv') 

# Remove duplicates & 'transactie' column
evides = evides.drop('Transactie', axis=1)
evides.drop_duplicates(inplace=True)

### Cleaning - Evides
First, let us assess the missing values.

In [ ]:
mv = pd.DataFrame(data=evides.isnull().sum(), columns=['Missing values'])
mv['% of total'] = round(mv['Missing values'] / len(evides) * 100, 2)
mv

The dataset does contain some missing values, but fortunately this affects only a very small percentage of observations. In addition to missing values, we'll also need to take into account values which are wrongly spelled or otherwise invalid. For example, the ENI-number should be equal to 8 arabic numbers. The 'Klant' column does contain lots of missing values. However, since this column is of no value for this research, it can be removed entirely. 

In [ ]:
evides = evides.drop('Klant', axis=1)
evides.dtypes

As we can see, ENI is currently stored as a string (object). ENI is a tricky column, for the following reasons.

* Some values start with a leading zero ('01234567'), whilst others don't ('1234567'). Regardless, these are still the same ENI IDs. 
* Some values contain less than 8 numbers ('123456')
* Some values contain weird formatting ('*1234567')
* Some values are missing (279 in total)
* Some values are misaligned (wrong ENI assigned to a shipment)

In order to resolve these issues, we take the following steps, in the following order.

1. We'll convert ENI values to numeric in order to get rid of the leading zero. This is done in order to make sure that there are no differences between ENI values for the same boats.
2. Since an ENI consists of 8 values by definition (or 7 here, since the leading zero was removed), we convert all ENI values which contain less than 7 characters to NaN. 
3. We now convert all NaN values to 0, to allow for int conversion.
4. We then convert the ENI column to integers to get rid of hidden .0's, 

In [ ]:
# 1. Convert ENI values to numeric
evides['ENI'] = pd.to_numeric(evides['ENI'], errors='coerce') # convert to numeric type

# 2. Convert invalid ENI values to NaN (affects 110 values)
evides['ENI'].loc[evides.ENI < 1000000] = 0

# Set other NaNs to 0 as well
evides['ENI'][evides['ENI'].isnull()] = 0

# 3. Convert to INT to get rid of hidden '.0'
evides['ENI'] = evides['ENI'].astype('int64', errors='ignore')

Now that we've done this, we can start looking at ENI numbers which are too small (6 characters). We then convert the column back to a string format. 

In [ ]:
print(len(evides[evides['ENI']==0]))
unresolved_indexes = []
for i in evides[evides['ENI'] == 0].index:
    # Get index of row with ENI = 0
    row = evides.loc[i]
    
    #print(row)

    #print(row)

    # Get other records with the same boat name and boat type 
    obj = evides[(evides['Scheepsnaam'] == row.Scheepsnaam) & (evides['Scheepstype']== row.Scheepstype)]
    
    #print(obj)
    
    if len(obj) == 0:
        unresolved_indexes.append(i)

    
    else:    
        # Group these other records by count of ENI
        grouped_obj = obj.groupby(['ENI']).size().sort_values(ascending=False)

        # Replace ENI by most frequently occuring ENI for this ship name and ship type
        row.ENI = grouped_obj.index[0]

        if row.ENI == 0:
            try:
                row.ENI = grouped_obj.index[1]

            except:
                unresolved_indexes.append(i)

        evides.loc[i] = row

        #print(row.ENI, grouped_obj[row.ENI])

evides = evides[~evides.index.isin(unresolved_indexes)]

Through this method, we were able to recover 262 out of 403 rows with a missing or wrongly formatted ENI value. We'll now convert the ENIs back to a string format. Since many ENIs now have a length of 7, we'll readd the leading zero's. 

In [ ]:
evides['ENI'] = evides['ENI'].astype('str')

for i in evides['ENI']:
    # if len == 7, then immediately add the leading 0 to all instances of this ENI. 
    if len(i) == 7:
        evides['ENI'][evides['ENI']==i] = '0' + i

evides['ENI']

Great, all ENIs now have the right length and format. Unfortunately, there are other issues. For example, the records below show observations that belong to the ship 'nijmegen max', with ENI 02338661. However, there are also two records with ship name 'atlantic prestige'. 

In [ ]:
evides[evides['ENI']=='02338661']

In [ ]:
evides[evides['Scheepsnaam']=='atlantic prestige']

We can see here that the actual ENI of the atlantic prestige is quite close to the ENI of the nijmegen max, yet it is slightly different. Let's see how often we get different names for the same ENI. 

In [ ]:
eni_data = pd.DataFrame(evides.groupby(['ENI','Scheepsnaam']).size())
eni_data = eni_data.add_suffix('_Count').reset_index()
eni_data.columns = ['ENI','Scheepsnaam','Count']

eni_data['Occurances'] = [len(eni_data[eni_data['ENI']==x]) for x in eni_data['ENI']]
eni_data

This table shows all ENIs, corresponding ship names and its counts. The column "occurances" shows the count of unique ENI IDs in this table. By subsetting for 'Occurances' > 1, we can find all ENIs that will need to be examined. 

In [ ]:
eni_data = eni_data[eni_data['Occurances'] > 1]
print("Unique ENI's left: {w}".format(w=len(set(eni_data['ENI']))))
eni_data

As it turns out, there are also many instances of ENI-Ship name combinations which are almost identical, but not fully. For example, 'fairplay 11' and 'fairplay XI' clearly refer to the same ship. We'll assume that vessels whose names differ only by 1, 2 or 3, 4 or 5 characters are the same name, but spelled differently. We can only do this for ENIs that occur two times, since we will be comparing two names. 

In [ ]:
# Create same_name column
eni_data['Same name'] = [999 for x in eni_data['ENI']]

# Fill it for occurances = 2
eni_data['Same name'][eni_data['Occurances']==2] = [jellyfish.damerau_levenshtein_distance(
    eni_data['Scheepsnaam'][eni_data['ENI']==x].values[0],
    eni_data['Scheepsnaam'][eni_data['ENI']==x].values[1]                        
    ) for x in eni_data['ENI'][eni_data['Occurances']==2]]

# Print it
eni_data.sort_values(by="Same name", ascending=True)

Now, let's get rid of all values for which "Same name" <= 5

In [ ]:
eni_data = eni_data[eni_data['Same name'] > 5]
print("Unique ENI's left: {w}".format(w=len(set(eni_data['ENI']))))
eni_data

There are now 195 unique ENI's left to check. This is still an incredibly large amount of work to check manually. Thus, we will write an algorithm that helps us solve this. 

In [ ]:
eni_data = eni_data.drop(['Occurances','Same name'], axis=1)
eni_data

In [ ]:
eni_data['Most frequently occuring name for this ENI'] = [eni_data[eni_data['ENI']==x].sort_values(by="Count", ascending=False).reset_index()['Scheepsnaam'][0] for x in eni_data['ENI']]
eni_data['Difference in characters'] = [jellyfish.damerau_levenshtein_distance(
    eni_data['Scheepsnaam'][x],
    eni_data['Most frequently occuring name for this ENI'][x]
) for x in eni_data.index]

eni_data = eni_data[eni_data['Difference in characters'] > 0]
eni_data.sort_values(by='Difference in characters', ascending=True)

We see that many of the names with small differences in characters are very similar. We can assume that these are the same boats. We'll filter based on difference in characters <= 3.

In [ ]:
eni_data = eni_data[eni_data['Difference in characters'] > 3]
print("Unique ENI's left: {w}".format(w=len(set(eni_data['ENI']))))
eni_data

184 ENI's left. Let's find out what the most frequently occuring ENI IDs are for these names. 

In [ ]:
eni_data['Most freq. occuring ENI for this ship name'] = [evides[['ENI','Scheepsnaam']][evides['Scheepsnaam']==x].groupby(by='ENI').size().sort_values(ascending=False).index[0] for x in eni_data['Scheepsnaam']]
eni_data['Corresponding count'] = [evides[['ENI','Scheepsnaam']][evides['Scheepsnaam']==x].groupby(by='ENI').size().sort_values(ascending=False)[0] for x in eni_data['Scheepsnaam']]
eni_data[['ENI','Scheepsnaam','Count','Most freq. occuring ENI for this ship name','Corresponding count']]

eni_data = eni_data[['ENI','Scheepsnaam','Count','Most freq. occuring ENI for this ship name', 'Corresponding count']][eni_data['ENI'] != eni_data['Most freq. occuring ENI for this ship name']]
eni_data

Great! As can be gathered from the table above, these ENIs are extremely similar yet slightly different. Using this table as our input, we'll replace the ENI's for these ENI-Ship name combinations with the most frequently occuring ENI for the corresponding ship name. 

In [ ]:
for i in eni_data.index:
    # Get information
    ENI = eni_data.loc[i]['ENI']
    Scheepsnaam = eni_data.loc[i]['Scheepsnaam']
    NEW_ENI = eni_data.loc[i]['Most freq. occuring ENI for this ship name']
    
    # Replace values
    evides['ENI'][(evides['ENI'] == ENI) & (evides['Scheepsnaam'] == Scheepsnaam)] = NEW_ENI

Now this is done, we can continue with the rest of the data cleaning. Since we'll be looking at time trends, we'll create a column called "Months" which tracks the month for every delivery.

In [ ]:
evides['Month'] = evides['Datum'].dt.strftime('%B')
evides['Month'] = pd.Categorical(evides['Month'], categories=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)
evides['Month_number'] = evides['Datum'].dt.strftime('%m')
evides = evides.sort_values(by='Datum')

Let's take a further look at the data. Since there are five boats in total, we'd expect five unique values in the ['Waterboot'] column. 

In [ ]:
set(evides['Waterboot'].values)

Yet, there is a sixth called 'wal'.

In [ ]:
evides[evides['Waterboot']=='wal']

It only concerns two observations, so we can remove them.

In [ ]:
evides = evides.loc[evides.Waterboot != 'wal']

So, what about the districts? These are stored in the 'Wijk' column. 

In [ ]:
set(evides['Wijk'].values)

There should only be four districts, but here there are five. 

In [ ]:
evides[evides['Wijk']==5]

Fortunately, it only concerns four observations. Upon further inspection, the locations of each of these shipments is in or near Dordrecht. For this research project, these locations are not relevant. Therefore, these observations will be removed. In addition, we will convert the 'Wijk' column datatype to 'category', since its values represent district names. 

In [ ]:
evides = evides.loc[evides.Wijk != 5]
evides['Wijk'] = evides['Wijk'].astype('object')
evides.dropna(inplace=True) # remove remaing NA values

Let us do one more final check.

In [ ]:
evides.dtypes

Great, now let's save our cleaned and formatted dataset. 

In [ ]:
evides.to_csv('../Data/Cleaned data/evides_cleaned.csv')